<h2>2:22:50 / 3:11:52</h2>

<h1>Workflow</h1>

<pre>
    0. Import important liberaries
    1. Get data ready (turn into tensors and batches)
    2. Build a logistic regression model
    3. Pick loss function and optimizer
    4. Build a training loop
    5. Evaluate your model
    6. How to improve our model
    7. Save your model
</pre>

<h2>0. Import important liberaries</h2>

In [89]:
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

<h2>1. Get data ready (turn into tensors and batches</h2>

<h3>Download datasets</h3>

In [90]:
train_dataset = datasets.FashionMNIST(root="/dataset", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.FashionMNIST(root="/dataset", train=False, transform=transforms.ToTensor())

In [91]:
train_dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: /dataset
    Split: Train
    StandardTransform
Transform: ToTensor()

In [92]:
test_dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: /dataset
    Split: Test
    StandardTransform
Transform: ToTensor()

<h2>2. Convert data into batches</h2>

In [93]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [94]:
len(train_loader) # 60000 / 64 = 938

938

In [95]:
len(test_loader) #10000 / 64 = 157

157

<h2>3. Build logistic model</h2>

In [96]:
class LogisticRegression(nn.Module):
    def __init__(self, in_dim, n_class):
        super().__init__()
        self.linear = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.linear(x)
        return out

In [97]:
model = LogisticRegression(28*28, 10) #28*28 = 784

In [98]:
model

LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)

In [99]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0035, -0.0194, -0.0144,  ...,  0.0071, -0.0303, -0.0003],
                      [ 0.0333, -0.0004, -0.0117,  ..., -0.0281,  0.0276,  0.0096],
                      [ 0.0108,  0.0245,  0.0265,  ...,  0.0259, -0.0179, -0.0350],
                      ...,
                      [-0.0172,  0.0129, -0.0060,  ..., -0.0288, -0.0327,  0.0144],
                      [-0.0167, -0.0194,  0.0331,  ...,  0.0124,  0.0064, -0.0158],
                      [-0.0319,  0.0171, -0.0208,  ..., -0.0095, -0.0254,  0.0216]])),
             ('linear.bias',
              tensor([-0.0303,  0.0221,  0.0133,  0.0122,  0.0178, -0.0345, -0.0252, -0.0257,
                      -0.0148,  0.0110]))])

In [100]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Code for GPU

In [101]:
device

'cuda'

In [102]:
model = model.to(device) #Send model to tun at GPU

<h2>4. Pick a loss function and optimizer</h2>

In [103]:
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [104]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0035, -0.0194, -0.0144,  ...,  0.0071, -0.0303, -0.0003],
         [ 0.0333, -0.0004, -0.0117,  ..., -0.0281,  0.0276,  0.0096],
         [ 0.0108,  0.0245,  0.0265,  ...,  0.0259, -0.0179, -0.0350],
         ...,
         [-0.0172,  0.0129, -0.0060,  ..., -0.0288, -0.0327,  0.0144],
         [-0.0167, -0.0194,  0.0331,  ...,  0.0124,  0.0064, -0.0158],
         [-0.0319,  0.0171, -0.0208,  ..., -0.0095, -0.0254,  0.0216]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-0.0303,  0.0221,  0.0133,  0.0122,  0.0178, -0.0345, -0.0252, -0.0257,
         -0.0148,  0.0110], device='cuda:0', requires_grad=True)]

In [105]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[-0.0035, -0.0194, -0.0144,  ...,  0.0071, -0.0303, -0.0003],
                      [ 0.0333, -0.0004, -0.0117,  ..., -0.0281,  0.0276,  0.0096],
                      [ 0.0108,  0.0245,  0.0265,  ...,  0.0259, -0.0179, -0.0350],
                      ...,
                      [-0.0172,  0.0129, -0.0060,  ..., -0.0288, -0.0327,  0.0144],
                      [-0.0167, -0.0194,  0.0331,  ...,  0.0124,  0.0064, -0.0158],
                      [-0.0319,  0.0171, -0.0208,  ..., -0.0095, -0.0254,  0.0216]],
                     device='cuda:0')),
             ('linear.bias',
              tensor([-0.0303,  0.0221,  0.0133,  0.0122,  0.0178, -0.0345, -0.0252, -0.0257,
                      -0.0148,  0.0110], device='cuda:0'))])

<h2>5. Building a training loop</h2>

In [106]:
num_epochs = 6
for epoch in range(num_epochs):
    print('* ' *10)
    print(f'epoch {epoch +1}')
    since = time.time()
    running_loss = 0.0
    running_accu = 0.0
    model.train()
    for i, data in enumerate(train_loader, 1): #937
        img, label = data
        img = img.view(img.size(0),-1)
        img = img.to(device)
        label = label.to(device)
        out = model(img) #forward pass
        loss = criterion(out, label)
        running_loss += loss.item()
        _,pred = torch.max(out, 1)
        running_accu += (pred == label).float().mean()
        optimizer.zero_grad()
        loss.backward() #backward pass
        optimizer.step()
        if i % 300 == 0:
            print(f'[{epoch+1} / {num_epochs}], loss: {running_loss / i:.6f}, accuracy: {running_accu/i:.6f}')
    print(f'Finish {epoch+1} epoch, loss: {running_loss/i:.6f}, accu: {running_accu/i:.6f}')

* * * * * * * * * * 
epoch 1
[1 / 6], loss: 2.014395, accuracy: 0.379792
[1 / 6], loss: 1.834694, accuracy: 0.491771
[1 / 6], loss: 1.703800, accuracy: 0.545382
Finish 1 epoch, loss: 1.690839, accu: 0.549157
* * * * * * * * * * 
epoch 2
[2 / 6], loss: 1.284526, accuracy: 0.672188
[2 / 6], loss: 1.240665, accuracy: 0.672161
[2 / 6], loss: 1.203443, accuracy: 0.673854
Finish 2 epoch, loss: 1.199102, accu: 0.674074
* * * * * * * * * * 
epoch 3
[3 / 6], loss: 1.065344, accuracy: 0.685365
[3 / 6], loss: 1.044368, accuracy: 0.688490
[3 / 6], loss: 1.025352, accuracy: 0.693507
Finish 3 epoch, loss: 1.023736, accu: 0.693980
* * * * * * * * * * 
epoch 4
[4 / 6], loss: 0.963111, accuracy: 0.701823
[4 / 6], loss: 0.943601, accuracy: 0.709453
[4 / 6], loss: 0.932389, accuracy: 0.711354
Finish 4 epoch, loss: 0.930929, accu: 0.711987
* * * * * * * * * * 
epoch 5
[5 / 6], loss: 0.888828, accuracy: 0.724063
[5 / 6], loss: 0.878667, accuracy: 0.727109
[5 / 6], loss: 0.872378, accuracy: 0.729410
Finish 